# Lecture 7 — Carbon Footprint Accounting Meets Firm Valuation

This notebook walks through `0209Lecture 6 - Firm Valuation .pdf` **page by page** and adds
**carbon-footprint (GHG) accounting** intuition plus **Python practice**.

## Learning goals
1. Connect Scope 1/2/3 emissions to cash-flow and discount-rate drivers.
2. Build a simple DCF model in Python from financial statements.
3. Map a carbon footprint to a carbon-cost scenario (carbon price × emissions).
4. Run sensitivity analysis (WACC, terminal growth, carbon price / intensity).
5. Discuss how carbon intensity can refine comparable-company selection.


In [1]:
from pathlib import Path

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')

# Network-dependent cells are optional.
# We use `akshare` (Sina/Baidu/Eastmoney data sources) instead of `yfinance` to avoid Yahoo rate limits.
RUN_ONLINE = False  # set True if you want to fetch live data

# Allow running online when executing with nbconvert, e.g.:
#   RUN_ONLINE=1 python -m nbconvert --execute ...
RUN_ONLINE = RUN_ONLINE or (os.environ.get('RUN_ONLINE', '0') == '1')

# This notebook lives under `materials/`. Depending on how Jupyter is launched,
# the kernel's working directory may be repo root or `materials/`.
DATA_DIR = Path('data') if Path('data').exists() else Path('materials/data')
SLIDE_DIR = (
    Path('pic') / '0209_lecture6_firm_valuation'
    if (Path('pic') / '0209_lecture6_firm_valuation').exists()
    else Path('materials/pic/0209_lecture6_firm_valuation')
)

DATA_DIR.exists(), SLIDE_DIR.exists()


(True, True)

## Page 01 — Firm Valuation

### Slide content

junjie.zhang@duke.edujunjie.zhang@duke.edu
Junjie Zhang
ENERGY 578K: ESG Investing
Duke Kunshan University
Firm Valuation

<details>
<summary>Slide screenshot (PDF page 1)</summary>

![](pic/0209_lecture6_firm_valuation/page-01.png)

</details>


## Page 02 — Tesla’s ESG Risks and Opportunities

### Slide content

Tesla’s ESG Risks and Opportunities

<details>
<summary>Slide screenshot (PDF page 2)</summary>

![](pic/0209_lecture6_firm_valuation/page-02.png)

</details>


## Page 03 — Tesla’s Impact Report

### Slide content

Tesla’s Impact Report
Environmental considerations
- Purpose of business: Accelerating global energy transition by designing and manufacturing a
complete energy and transportation ecosystem
- Scope 1&2: More efficient and sustainable factories
- Scope 3: Reducing supply-chain carbon footprint
Social considerations
- Child or forced labor and human trafficking (OECD Due Diligence Guidelines)
- Human capital resources
Corporate governance considerations
- Board of Directors, Sustainability and Impact team
3
Source: Annual Report of Tesla
Source: Annual Report of Tesla

<details>
<summary>Slide screenshot (PDF page 3)</summary>

![](pic/0209_lecture6_firm_valuation/page-03.png)

</details>


### Carbon-footprint mini-practice: activity data → emissions

A corporate carbon footprint is typically estimated as:

- **Emissions = activity × emission factor**

Below is a tiny toy example to show the mechanics.


In [2]:
activity = pd.DataFrame(
    {
        'Activity': ['Natural gas (Scope 1)', 'Purchased electricity (Scope 2)', 'Inbound logistics (Scope 3)'],
        'Quantity': [8_000, 120_000, 5_500],
        'Unit': ['MMBtu', 'MWh', 'ton-km'],
        # Emission factors are illustrative only (do not use for reporting).
        'EF (tCO2e/unit)': [0.053, 0.0004, 0.00008],
    }
)
activity['Emissions (tCO2e)'] = activity['Quantity'] * activity['EF (tCO2e/unit)']
activity


,Activity,Quantity,Unit,EF (tCO2e/unit),Emissions (tCO2e)
0,Natural gas (Scope 1),8000,MMBtu,0.05300,424.00
1,Purchased electricity (Scope 2),120000,MWh,0.00040,48.00
2,Inbound logistics (Scope 3),5500,ton-km,0.00008,0.44


In [3]:
total = activity['Emissions (tCO2e)'].sum()
activity.assign(Share=activity['Emissions (tCO2e)'] / total).sort_values(
    'Emissions (tCO2e)', ascending=False
)


,Activity,Quantity,Unit,EF (tCO2e/unit),Emissions (tCO2e),Share
0,Natural gas (Scope 1),8000,MMBtu,0.05300,424.00,0.897468
1,Purchased electricity (Scope 2),120000,MWh,0.00040,48.00,0.101600
2,Inbound logistics (Scope 3),5500,ton-km,0.00008,0.44,0.000931


## Page 04 — ESG Opportunity: Trading the Automotive Regulatory Credits

### Slide content

ESG Opportunity: Trading the Automotive Regulatory Credits
Zero Emission Vehicle (ZEV) Requirements
- Automakers must sell a certain percentage of zero-emission vehicles (EVs, hydrogen fuel cells)
- Adopted by California and other states
Corporate Average Fuel Economy (CAFE) Standards
- Automakers must ensure their fleet meets certain fuel efficiency standards for cars and light trucks
- National Highway Traffic Safety Administration (NHTSA)
Greenhouse Gas (GHG) Emission Standards
- Automakers must meet annual fleet-wide average emission limits (g CO₂/mile)
- Environmental Protection Agency (EPA)
Renewable Fuel Standards (RFS)
- Transportation fuels sold in the U.S. must contain a minimum volume of renewable fuels
- Environmental Protection Agency (EPA)

<details>
<summary>Slide screenshot (PDF page 4)</summary>

![](pic/0209_lecture6_firm_valuation/page-04.png)

</details>


## Page 05 — ZEV Credits in California

### Slide content

5
ZEV Credits in California
Note: AT PZEV (advanced technology partial zero emission vehicle), BEV (battery electric vehicle), NEV+ (neighborhood electric vehicle), PHEV (plug-in
hybrid electric vehicle), PZEV (partial zero emission vehicle); FCVs (hydrogen fuel cell vehicles).
Source: https://ww2.arb.ca.gov/applications/annual-zev-credits-disclosure-dashboard
ZEV sales requirements
- Automakers’ requirements of ZEV sales
proportional to total vehicle sales in CA
Credit allocation
- Automakers earn credits for each ZEV
or PZEV depending on range and
technology
- BEVs and FCVs earn the most credits;
PHEVs earn fewer credits)
Compliance
- Trading and banking of credits
- Financial penalty for incompliance
ZEV credits transfer, Model Year 2023

<details>
<summary>Slide screenshot (PDF page 5)</summary>

![](pic/0209_lecture6_firm_valuation/page-05.png)

</details>


## Page 06 — Impacts of Tradable Credits on Tesla

### Slide content

Impacts of Tradable Credits on Tesla
6
Tesla earns tradable credits from federal and state
regulatory programs such as ZEV, GHG, fuel economy,
and clean fuel.
- The cost of automotive regulatory credits is
“negligible.” Profit from automotive regulatory credits
(2025):
1,993 million USD
- Impact on the net income:
1,993 / 3,855 = 51.6%
- Changes in regulation on automotive regulatory
credits at the federal and state levels can
significantly affect Tesla's financial performance.
Source: Annual Report of Tesla
Source: Annual Report of Tesla

<details>
<summary>Slide screenshot (PDF page 6)</summary>

![](pic/0209_lecture6_firm_valuation/page-06.png)

</details>


## Page 07 — Sensitivity Analysis on Tradable Credits

### Slide content

Sensitivity Analysis on Tradable Credits
7
Tesla has significantly benefited from Tradable Credits, which have contributed to its profitability and cash flow. These
credits are earned when Tesla produces zero-emission vehicles and can be sold to other automakers that do not
meet emission standards. However, this might be a non-recurring revenue source, raising an important question:
What if Tesla is forced to discontinue its participation in the Tradable Credits market?”
Let’s first consider the impact of canceling Tradable Credits on Tesla’s financial statement items.
Impact on the Income Statement
Impact on the Balance Sheet
Impact on the Cash Flow Statement
“Regulatory Credit Sales” (Under
Revenue) Decrease
“Cash and Cash Equivalents” (Under
Asset) Decrease
“Cash from Regulatory Credit Sales”
Decrease
“Retained Earnings” (Under Equity)
Decrease
These are the intuitive losses for Tesla, however, there might be some benefits which are somekind complicated :
- More predictable and recurring revenue sources in EVs, improving Tesla’s financial stability.
- Investors will have more confidence in Tesla’s core profitability rather than policy-driven earnings.
- Tesla becomes less reliant on external government incentives for liquidity.

<details>
<summary>Slide screenshot (PDF page 7)</summary>

![](pic/0209_lecture6_firm_valuation/page-07.png)

</details>


## Page 08 — Results

### Slide content

Results
8
Profitability
Before
After
Liquidity
Before After
Efficiency
Before
After
Gross Margin
= Revenue −COGS
Revenue
18.03%16.27%
Current Ratio
=
Current Assets
Current Liabilities
2.16
2.10
Inventory Turnover
=
COGS
Average Inventory
6.37
No
Impacts
Operating Profit Margin
 = Operating Profit EBIT
Revenue
5.92% 3.90%
Quick Ratio
= Current Assets −Inventory
Current Liabilities
1.77
1.71
Accounts Payable Turnover
=
COGS
Average Accounts Payable
6.02
No
Impacts
Net Profit Margin
= Net Income
Revenue
4.00% 1.94%
Cash Ratio
= Cash +  Cash Equivalents
Current Liabilities
0.52
0.46
Accounts Receivable Turnover
=
Net Credit Sales
Average Accounts Receivable
21.09
20.64
A simple version:
- We assume that all tradable credit revenue in 2025 ($ 1,993 million) is fully eliminated, with no
immediate short-term impact on consumer demand.

<details>
<summary>Slide screenshot (PDF page 8)</summary>

![](pic/0209_lecture6_firm_valuation/page-08.png)

</details>


### Python practice: reproduce the “remove credits” ratio changes

The slides remove **$1,993M** of regulatory credit revenue and assume it has ~100% margin.
We can reproduce the before/after ratios from the cached statements.


In [4]:
income = pd.read_excel(DATA_DIR / 'tesla_income_stmt.xlsx', index_col=0)
cashflow = pd.read_excel(DATA_DIR / 'tesla_cash_flow.xlsx', index_col=0)
balance = pd.read_excel(DATA_DIR / 'tesla_balance_sheet.xlsx', index_col=0)

base_year = income.index.max()
prev_year = income.index.sort_values()[-2]

base_year


Timestamp('2025-12-31 00:00:00')

In [5]:
CREDIT_MUSD = 1993
credit = CREDIT_MUSD * 1e6

rev = float(income.loc[base_year, 'Total Revenue'])
cogs = float(income.loc[base_year, 'Cost Of Revenue'])
gross_profit = float(income.loc[base_year, 'Gross Profit'])
ebit = float(income.loc[base_year, 'EBIT'])
net_income = float(income.loc[base_year, 'Net Income From Continuing Operation Net Minority Interest'])

current_assets = float(balance.loc[base_year, 'Current Assets'])
current_liabilities = float(balance.loc[base_year, 'Current Liabilities'])
inventory = float(balance.loc[base_year, 'Inventory'])
cash_eq = float(balance.loc[base_year, 'Cash And Cash Equivalents'])

avg_inventory = (float(balance.loc[base_year, 'Inventory']) + float(balance.loc[prev_year, 'Inventory'])) / 2
avg_ap = (float(balance.loc[base_year, 'Accounts Payable']) + float(balance.loc[prev_year, 'Accounts Payable'])) / 2
avg_ar = (float(balance.loc[base_year, 'Accounts Receivable']) + float(balance.loc[prev_year, 'Accounts Receivable'])) / 2

def ratios(rev_, gross_profit_, ebit_, net_income_, cash_eq_, current_assets_):
    return {
        'Gross Margin': gross_profit_ / rev_,
        'Operating Profit Margin (EBIT)': ebit_ / rev_,
        'Net Profit Margin': net_income_ / rev_,
        'Current Ratio': current_assets_ / current_liabilities,
        'Quick Ratio': (current_assets_ - inventory) / current_liabilities,
        'Cash Ratio': cash_eq_ / current_liabilities,
        'Inventory Turnover': cogs / avg_inventory,
        'Accounts Payable Turnover': cogs / avg_ap,
        'Accounts Receivable Turnover': rev_ / avg_ar,
    }

before = ratios(rev, gross_profit, ebit, net_income, cash_eq, current_assets)
after = ratios(
    rev - credit,
    gross_profit - credit,
    ebit - credit,
    net_income - credit,
    cash_eq - credit,
    current_assets - credit,
)

summary = pd.DataFrame({'Before': before, 'After': after})
pct = ['Gross Margin', 'Operating Profit Margin (EBIT)', 'Net Profit Margin']
summary.loc[pct] = (summary.loc[pct] * 100)
summary.round(2)


,Before,After
Gross Margin,18.03,16.27
Operating Profit Margin (EBIT),5.92,3.90
Net Profit Margin,4.00,1.94
Current Ratio,2.16,2.10
Quick Ratio,1.77,1.71
Cash Ratio,0.52,0.46
Inventory Turnover,6.37,6.37
Accounts Payable Turnover,6.02,6.02
Accounts Receivable Turnover,21.09,20.64


## Page 09 — Valuation

### Slide content

Valuation

<details>
<summary>Slide screenshot (PDF page 9)</summary>

![](pic/0209_lecture6_firm_valuation/page-09.png)

</details>


## Page 10 — Valuation

### Slide content

Valuation
- Valuation is the process of estimating the economic worth of a firm, investment, or asset based on
its fundamentals, including expected future cash flows and financial performance.
o While market prices reflect current trading conditions, valuation assesses intrinsic value—the
estimated worth derived from underlying economic drivers.
- Valuation plays a central role in corporate finance, investment analysis, and strategic decision-
making by providing a basis for assessing whether an asset or business is overvalued or
undervalued relative to its underlying economic value.
- Firm value and equity value are two different concepts: Equity Value = Firm Value - Net Debt
10
Firm value
- Market value of all capital invested in a business (also referred to as “enterprise
value” or “asset value”)
- The value of the total enterprise: market value of equity + net debt
Equity value •
Market value of the shareholders’ equity (also referred to as “offer value”)
- The market value of a company’s equity (shares outstanding x current stock price)

<details>
<summary>Slide screenshot (PDF page 10)</summary>

![](pic/0209_lecture6_firm_valuation/page-10.png)

</details>


## Page 11 — Reasons for Performing a Valuation

### Slide content

Reasons for Performing a Valuation
11
Valuation Purposes
Explanation
Transaction
and Market
Decisions
Buying or Selling a
Business
Valuation is used to estimate a fair transaction price by
assessing the firm’s financial position and future cash-flow
potential
Securities Investing
Valuation helps investors evaluate whether securities are
mispriced relative to intrinsic value and comparable assets.
Strategic
Decisions
Strategic Planning
Valuation supports capital allocation decisions by identifying
projects, acquisitions, or expansion strategies that are expected
to increase firm value.
Capital Financing
Valuation is applied to assess a firm’s capacity to generate
cash flows and service debt, providing a basis for lending
and investment decisions.
Regulatory,
Accounting,
Compliance
Tax Purposes and
Financial Reporting
Valuation is required for fair value measurement in regulatory
compliance, taxation, estate planning, and financial reporting.

<details>
<summary>Slide screenshot (PDF page 11)</summary>

![](pic/0209_lecture6_firm_valuation/page-11.png)

</details>


## Page 12 — Core Valuation Approaches

### Slide content

Core Valuation Approaches
12
- Market-based, public
equity markets
- Market approach
- Firm value is inferred by
applying trading
multiples of comparable
publicly listed companies
to the target firm’s
financial metrics
- IPO pricing, equity
research, and cross-
check for DCF
- Private market valuation
with control consideration
- Market approach
- Firm value is estimated
using valuation
multiples observed in
comparable M&A
transactions
- M&A negotiations,
strategic sales, fairness
opinions
Public Trading
Comparables Analysis
Precedent
Transactions Analysis
- Intrinsic, fundamentals-
based valuation
- Income approach
- Firm value is estimated
as the present value of
projected free cash
flows, discounted at a
rate reflecting operating
and financing risk
- M&A valuation, fairness
opinions, long-term
investment analysis
Discounted Cash Flow
(DCF)
- Value to a financial
sponsor.
- Income approach
- Firm value is determined
based on projected cash
flows and debt service
capacity, which drive
equity returns (IRR).
- Private equity
acquisitions,
recapitalizations, and
sponsor-led transactions.
Leveraged Buyout
(LBO) Analysis
Source: J.P. Morgan

<details>
<summary>Slide screenshot (PDF page 12)</summary>

![](pic/0209_lecture6_firm_valuation/page-12.png)

</details>


## Page 13 — Valuation Framework

### Slide content

Valuation Framework
13
Data Import
Review historical financial statements to understand
the firm’s recent financial performance
Assumption Setting for Forecasting
- Revenue: Forecast future revenues and costs (or
growth rates), informed by capacity, sales plans, market
share, and industry cycles.
- Investment: Input projected capital expenditures and
investment schedules, considering capital intensity and
expansion plans.
- Financing: Forecast future financing needs, including
debt and equity issuance, based on investment plans
and funding constraints.
Import historical financial data
Financial Statement Projection: Investment, financing,
and operating activities are internally linked
Forecast Rationality Checks
Intrinsic (Absolute) Valuation
- Set key valuation assumptions
- Execute valuation calculations
- Valuation model: DCF
Sensitivity Analysis
- Generate sensitivity matrices showing how
intrinsic value responds to changes in:
o
WACC
o
Terminal growth rate
Comparable Company Analysis
- Select industry classification and the peer list.
- Calculate benchmark valuation multiples.
- Compute value estimates using: P/E, P/S, P/B
Valuation Output
Source: McKinsey

<details>
<summary>Slide screenshot (PDF page 13)</summary>

![](pic/0209_lecture6_firm_valuation/page-13.png)

</details>


## Page 14 — Discounted Cash Flow Analysis (DCF)

### Slide content

Discounted Cash Flow Analysis (DCF)
Free cash flow (FCF) model
𝑉= ෍
𝑡=1
𝑛
𝐹𝐶𝐹𝑡
(1 + 𝑊𝐴𝐶𝐶)𝑡+
𝐹𝐶𝐹𝑛+1
(𝑊𝐴𝐶𝐶−𝑔)(1 + 𝑔)

1. Forecasting the firm’s free cash flows for each year of the explicit projection horizon:
𝐹𝐶𝐹= 𝐸𝐵𝐼𝑇1 + 𝑡+ 𝐷𝑒𝑝𝑟𝑒𝑐𝑖𝑎𝑡𝑖𝑜𝑛−𝐶𝑎𝑝𝑖𝑡𝑎𝑙 𝑒𝑥𝑝𝑒𝑛𝑑𝑖𝑡𝑢𝑟𝑒−∆ 𝑊𝑜𝑟𝑘𝑖𝑛𝑔 𝐶𝑎𝑝𝑖𝑡𝑎𝑙
2. Estimate the appropriate discount rate:
- The weighted average cost of capital (WACC) should be commensurate with the riskiness of the project
𝑊𝐴𝐶𝐶= 𝑟𝑒× 𝐸
𝑉+ 𝑟𝑑(1 + 𝑡) × 𝐷
𝑉
- Where: 𝑟𝑒= 𝑐𝑜𝑠𝑡 𝑜𝑓 𝑒𝑞𝑢𝑖𝑡𝑦; 𝑟𝑑= 𝑐𝑜𝑠𝑡 𝑜𝑓 𝑑𝑒𝑏𝑡; 𝐸= 𝑡𝑜𝑡𝑎𝑙 𝑒𝑞𝑢𝑖𝑡𝑦; 𝐷= total debt; V = E + D
3. Estimate the Terminal Value: using a long-term growth assumption to reflect cash flows beyond the
explicit projection horizon.
14
Projection Period
Terminal value
(growing perpetuity method)

<details>
<summary>Slide screenshot (PDF page 14)</summary>

![](pic/0209_lecture6_firm_valuation/page-14.png)

</details>


### DCF recap (clean formulas)

We’ll use the standard operating FCF definition (firm value):

$$
FCF_t = EBIT_t(1-T) + D\&A_t - CapEx_t - \Delta NWC_t
$$

and discount with WACC:

$$
V = \sum_{t=1}^{n} \frac{FCF_t}{(1+WACC)^t} + \frac{TV_n}{(1+WACC)^n}
$$


In [6]:
def dcf_pv(cashflows, wacc):
    cashflows = np.asarray(cashflows, dtype=float)
    t = np.arange(1, len(cashflows) + 1)
    return (cashflows / (1 + wacc) ** t).sum()

def terminal_value_growing_perpetuity(fcf_t, wacc, g):
    if g >= wacc:
        raise ValueError('g must be smaller than WACC for a stable perpetuity.')
    return fcf_t * (1 + g) / (wacc - g)


## Page 15 — Projecting the Free Cash Flows

### Slide content

Projecting the Free Cash Flows
15
Industry
outlook
Competitive
position
Ongoing
reinvestment
needs
Expansion
opportunities
Sources of
information
- Anticipated industry growth (supply versus demand)
- Major opportunities and risks; How does this impact on the company?
- Pricing flexibility
- Possible market share changes
- Cost structure
- Working capital
- Required capital expenditures
- Discretionary investments
- New product lines and Greenfield expansions
- Development costs
- Economies of scale
- Company and competitor reports/presentations
- Equity research analyst reports and industry sector reports
- Other (e.g. market data such as forward curves, regulators
Source: J.P. Morgan

<details>
<summary>Slide screenshot (PDF page 15)</summary>

![](pic/0209_lecture6_firm_valuation/page-15.png)

</details>


## Page 16 — Estimate the Discount Rate: the Cost of Equity

### Slide content

Estimate the Discount Rate: the Cost of Equity
Weighted average cost of capital (WACC):
𝑊𝐴𝐶𝐶= 𝑟𝑒× 𝐸
𝑉+ 𝑟𝑑(1 + 𝑡) × 𝐷
𝑉
- The cost of equity 𝒓𝒆 represents the long-term return expected by the market for this project. Industry wide
use of the capital asset pricing model (CAPM)
16
Cost of equity:
Long term return on
equity investment
Risk free rate:
Long term risk-free
rate of return
Beta:
Measures a firm’s
sensitivity to overall
market movements
Risk premium:
Extra market return
above risk free rate
=
+
×
Theoretical definition
𝛽= 𝐶ov(𝑅𝑖, 𝑅𝑚)
𝑉𝑎𝑟(𝑅𝑚)
Where: 𝑅𝑖 = is the firm’s
return and 𝑅𝑚= the
market return.
Beta can be defined and estimated
using the following approaches
Historical regression
Regressing the firm’s
historical stock returns on
market returns; the
regression slope equals
beta
Industry (bottom-up)
averaging unlevered betas
of comparable firms and
relevering them to reflect
the target firm’s capital
structure.
Database-reported
Obtained from financial
data providers (e.g.,
Bloomberg, Refinitiv,
Yahoo Finance)

<details>
<summary>Slide screenshot (PDF page 16)</summary>

![](pic/0209_lecture6_firm_valuation/page-16.png)

</details>


## Page 17 — Estimate the Discount Rate: the Cost of Debt

### Slide content

Estimate the Discount Rate: the Cost of Debt
17
Source: TRADING ECONOMICS
The cost of debt 𝒓𝒅:
𝑟𝑑= (𝑏+ 𝑠) × (1 −𝑡)
Benchmark rate
Domestic debt;
10-year government bonds
Debt spread
Bond yield spread = Corporate bond yield −
Government bond yield (same maturity)
Credit-rating-implied spread = the average
market spread associated with the firm’s
credit rating when bond data are unavailable
Loan markup = the interest rate premium
over the benchmark rate for bank loans
Marginal tax rate
Firm’s statutory/effective tax rate
10-Year Government Bond Yield
Proxy
Indictors

<details>
<summary>Slide screenshot (PDF page 17)</summary>

![](pic/0209_lecture6_firm_valuation/page-17.png)

</details>


## Page 18 — Estimate the Terminal Value

### Slide content

Estimate the Terminal Value
- When firm has reached a steady state
o Growth in perpetuity method
𝑔= (1 −𝐷𝑃𝑅) × 𝑅𝑂𝐸
𝑇𝑉𝑡= 𝐹𝐶𝐹𝑡× (1 + 𝑔)
𝑊𝐴𝐶𝐶−𝑔
Where, 𝑔= perpetual growth rate; 𝐷𝑃𝑅= dividend payout ratio; 𝑅𝑂𝐸= return on equity; 𝑇𝑉𝑡= terminal value at time 𝑡
- When firm has not reached a steady state
o Exit multiple approach: Terminal value is estimated by multiplying the target firm’s projected terminal-year
financial metric by valuation multiples derived from comparable firms.
𝑇𝑉𝑡= 𝐸𝑥𝑖𝑡 𝑀𝑢𝑙𝑡𝑖𝑝𝑙𝑒𝑡 × 𝐹𝑖𝑛𝑎𝑛𝑐𝑖𝑎𝑙 𝑀𝑒𝑡𝑟𝑖𝑐𝑡
o Common exit multiples
▪EV/EBITDA – used for firms with stable operating cash flows and varying capital structures
▪EV/EBIT – used for when depreciation and capital intensity are comparable across firms
▪EV/Revenue – used for high-growth or pre-profit firms
18
Notes: EV = Enterprise value; EBITDA = Operating earnings before interest, taxes, depreciation, and amortization;
EBIT = Operating earnings before interest and taxes, after depreciation and amortization.

<details>
<summary>Slide screenshot (PDF page 18)</summary>

![](pic/0209_lecture6_firm_valuation/page-18.png)

</details>


## Page 19 — Intrinsic Valuation of Tesla

### Slide content

Intrinsic Valuation of Tesla
Valuation Framework
- Primary method: Discounted Cash Flow
- Cross-check: Relative valuation using trading multiples
- Data source: Yahoo Finance
Collect historical financial data to establish the operating and cash flow base
- Key data
o Income Statement: Revenue, EBIT, Tax expense
o Cash Flow Statement: Depreciation & amortization (D&A), Capital expenditures (CapEx)
o Balance Sheet: Change in working capital (ΔNWC), Cash, and debt balances
- Historical ratios
o EBIT margin, D&A/Revenue, CapEx/Revenue, ΔNWC/Revenue
- Assumption
o Historical financials reflect Tesla’s normalized operating structure.

<details>
<summary>Slide screenshot (PDF page 19)</summary>

![](pic/0209_lecture6_firm_valuation/page-19.png)

</details>


## Page 20 — Project Future Cash Flows

### Slide content

Project Future Cash Flows
𝑭𝑪𝑭= 𝑬𝑩𝑰𝑻𝟏+ 𝒕+ 𝑫𝒆𝒑𝒓𝒆𝒄𝒊𝒂𝒕𝒊𝒐𝒏−𝑪𝒂𝒑𝒊𝒕𝒂𝒍 𝒆𝒙𝒑𝒆𝒏𝒅𝒊𝒕𝒖𝒓𝒆−∆ 𝑾𝒐𝒓𝒌𝒊𝒏𝒈 𝑪𝒂𝒑𝒊𝒕𝒂𝒍
- Analyst revenue growth expectations (Yahoo Finance input)
- Analyst EPS trends reflect market expectations of future profitability (used to guide margin assumptions)
- Forecast assumptions:
o No major structural change in business model during forecast period
o Depreciation and amortization are projected as a percentage of revenue (D&A/Revenue) based on historical
averages.
o Capital expenditures are forecast using historical CapEx/Revenue ratios, with a gradual convergence toward
depreciation in the steady state.
o Changes in working capital (∆𝑊𝐶𝑡) are estimated as a function of incremental revenue, reflecting stable
operating working capital requirements.
20
∆𝑊𝐶𝑡= (𝑅𝑒𝑣𝑒𝑛𝑢𝑒𝑡−𝑅𝑒𝑣𝑒𝑛𝑢𝑒𝑡−1) × (
𝑊𝐶
𝑅𝑒𝑣𝑒𝑛𝑢𝑒)ℎ𝑖𝑠𝑡𝑜𝑟𝑖𝑐𝑎𝑙 𝑣𝑎𝑙𝑢𝑒

<details>
<summary>Slide screenshot (PDF page 20)</summary>

![](pic/0209_lecture6_firm_valuation/page-20.png)

</details>


## Page 21 — Estimate the Discount Rate (WACC)

### Slide content

Estimate the Discount Rate (WACC)
1. Cost of Equity
𝑟𝑒= 𝑟𝑓+ 𝛽× 𝐸𝑅𝑃
- Beta: Yahoo Finance input
- Risk-free rate (𝑟𝑓): U.S. 10-year Treasury yield
- Equity risk premium (𝐸𝑅𝑃): the average excess
return of the S&P 500 over the risk-free rate
2. Capital Structure
- Yahoo Finance inputs: Market capitalization (𝑽),
Total debt (𝑫)
- Assumption: Current capital structure approximates
long-run target structure
21
3. Cost of Debt
- Market yield (YTM) on Tesla’s outstanding corporate
bonds
𝑾𝑨𝑪𝑪= 𝒓𝒆× 𝑬
𝑽+ 𝒓𝒅(𝟏+ 𝒕) × 𝑫
𝑽
Picture source: TradingView
- Tesla has multiple outstanding corporate bonds with
different yields. 𝒓𝒅 is estimated with a market-value-
weighted average.

<details>
<summary>Slide screenshot (PDF page 21)</summary>

![](pic/0209_lecture6_firm_valuation/page-21.png)

</details>


## Page 22 — Estimate Terminal Value

### Slide content

Estimate Terminal Value
Yahoo Finance Inputs:
- Payout ratio = 0% and ROE = 4.94% (in 2025FY)
- A zero-payout ratio indicates continued
reinvestment and expansion. Therefore, Tesla is in a
transition phase now rather than a fully steady-state.
Growth in perpetuity method
𝑔= (1 −𝐷𝑃𝑅) × 𝑅𝑂𝐸
𝑇𝑉𝑡= 𝐹𝐶𝐹𝑡× (1 + 𝑔)
𝑊𝐴𝐶𝐶−𝑔
Assumptions:
- Tesla is assumed to reach a steady state by the end of
the explicit forecast period in 2030.
- Terminal growth rate ≤ long-term nominal GDP growth
22
United States GDP Growth Rate
Picture source: TRADINGECONOMICS
4.4%

<details>
<summary>Slide screenshot (PDF page 22)</summary>

![](pic/0209_lecture6_firm_valuation/page-22.png)

</details>


### Python practice: a compact Tesla DCF + carbon adjustment

We now implement a *teaching* DCF model that follows the slide logic:

- forecast revenue, EBIT margin, D&A/revenue, CapEx/revenue
- estimate ΔWC from incremental revenue
- discount by WACC and compute a growing-perpetuity terminal value

Then we connect carbon footprint → valuation by subtracting an after-tax **carbon cost** scenario.


In [7]:
# Base-year ratios for forecasting
tax_rate = float(income.loc[base_year, 'Tax Rate For Calcs'])
rev0 = float(income.loc[base_year, 'Total Revenue'])
ebit0 = float(income.loc[base_year, 'EBIT'])
da0 = float(cashflow.loc[base_year, 'Depreciation'])
capex0 = float(cashflow.loc[base_year, 'Capital Expenditure'])

# Working capital in the FCF formula typically refers to *operating* NWC,
# excluding cash and financing items.
current_assets = float(balance.loc[base_year, 'Current Assets'])
current_liabilities = float(balance.loc[base_year, 'Current Liabilities'])
cash_like = float(balance.loc[base_year, 'Cash Cash Equivalents And Short Term Investments'])
current_debt = float(balance.loc[base_year, 'Current Debt'])
op_wc0 = (current_assets - cash_like) - (current_liabilities - current_debt)

ebit_margin = ebit0 / rev0
da_ratio = da0 / rev0
capex_ratio = (-capex0) / rev0
op_wc_ratio = op_wc0 / rev0

# Simple revenue-growth assumptions (edit freely)
forecast_years = [2026, 2027, 2028, 2029, 2030]
revenue_growth = [0.07, 0.06, 0.05, 0.04, 0.035]

revenue = [rev0]
for g in revenue_growth:
    revenue.append(revenue[-1] * (1 + g))
revenue = np.array(revenue[1:])

proj = pd.DataFrame({'Year': forecast_years, 'Revenue': revenue})
proj['EBIT'] = proj['Revenue'] * ebit_margin
proj['NOPAT'] = proj['EBIT'] * (1 - tax_rate)
proj['D&A'] = proj['Revenue'] * da_ratio
proj['CapEx'] = proj['Revenue'] * capex_ratio
proj['ΔWC'] = (proj['Revenue'].diff().fillna(proj['Revenue'].iloc[0] - rev0)) * op_wc_ratio
proj['FCF'] = proj['NOPAT'] + proj['D&A'] - proj['CapEx'] - proj['ΔWC']

proj_view = proj.assign(
    **{
        'Revenue ($B)': proj['Revenue'] / 1e9,
        'ΔWC ($B)': proj['ΔWC'] / 1e9,
        'FCF ($B)': proj['FCF'] / 1e9,
    }
)[['Year', 'Revenue ($B)', 'ΔWC ($B)', 'FCF ($B)']]
proj_view.round(2)


,Year,Revenue ($B),ΔWC ($B),FCF ($B)
0,2026,101.46,-0.39,2.23
1,2027,107.55,-0.36,2.31
2,2028,112.93,-0.32,2.36
3,2029,117.45,-0.26,2.40
4,2030,121.56,-0.24,2.45


In [8]:
# WACC (teaching assumptions)
price = pd.read_excel(DATA_DIR / 'tesla_daily_price.xlsx', index_col=0)
latest_close = float(price['Close'].iloc[-1])
shares = float(balance.loc[base_year, 'Ordinary Shares Number'])
market_cap = shares * latest_close

total_debt = float(balance.loc[base_year, 'Total Debt'])
cash_eq = float(balance.loc[base_year, 'Cash And Cash Equivalents'])
net_debt = total_debt - cash_eq
V = market_cap + total_debt

rf = 0.04
beta = 2.0
erp = 0.05
re = rf + beta * erp
rd_pre_tax = 0.06
rd = rd_pre_tax * (1 - tax_rate)
wacc = re * (market_cap / V) + rd * (total_debt / V)

wacc


0.13917642097924557

In [9]:
# Terminal growth (cap at ~4%)
equity = balance['Common Stock Equity'].dropna().sort_index()
ni = income['Net Income From Continuing Operation Net Minority Interest'].dropna().sort_index()
roe_2025 = float(ni.loc[base_year] / ((equity.loc[base_year] + equity.loc[prev_year]) / 2))
g_terminal = float(np.clip(roe_2025, 0.0, 0.04))

g_terminal


0.04

In [10]:
# Baseline valuation
fcf = proj['FCF'].to_numpy()
tv = terminal_value_growing_perpetuity(fcf_t=fcf[-1], wacc=wacc, g=g_terminal)
enterprise_value = dcf_pv(fcf, wacc) + tv / (1 + wacc) ** len(fcf)
equity_value = enterprise_value - net_debt
intrinsic_price = equity_value / shares

{'EV': enterprise_value, 'Equity value': equity_value, 'USD/share': intrinsic_price}


{'EV': np.float64(21409121996.47635),
 'Equity value': np.float64(23203121996.47635),
 'USD/share': np.float64(6.185849639156585)}

In [11]:
# Carbon adjustment (illustrative): carbon cost = (revenue × intensity) × carbon price
ei = 12.0  # tCO2e per $1M revenue (Scope 1+2 proxy)
carbon_price = 100.0  # $/tCO2e

revenue_musd = proj['Revenue'] / 1e6
carbon_cost = revenue_musd * ei * carbon_price
fcf_carbon = fcf - carbon_cost.to_numpy() * (1 - tax_rate)

enterprise_value_carbon = dcf_pv(fcf_carbon, wacc) + terminal_value_growing_perpetuity(
    fcf_t=fcf_carbon[-1], wacc=wacc, g=g_terminal
) / (1 + wacc) ** len(fcf_carbon)
equity_value_carbon = enterprise_value_carbon - net_debt
price_carbon = equity_value_carbon / shares

{'USD/share (baseline)': intrinsic_price, 'USD/share (carbon-adjusted)': price_carbon}


{'USD/share (baseline)': np.float64(6.185849639156585),
 'USD/share (carbon-adjusted)': np.float64(5.941597888899622)}

## Page 23 — Sensitivity Analysis

### Slide content

Sensitivity Analysis
23
更换为Tesla的DCF结果

<details>
<summary>Slide screenshot (PDF page 23)</summary>

![](pic/0209_lecture6_firm_valuation/page-23.png)

</details>


## Page 24 — Types of multiples

### Slide content

Types of multiples
Multiple
Comment
Firm value / sales
- Essential for high-tech companies
Firm value / EBITDA
- Good ratio in cyclical industries
- Good for cross-country comparisons
- Independent of leverage
Firm value / EBIT
- Useful when assessing a capital-intensive business
Market cap. / Net income
(also known as P/E)
- Historical P/E ratio – affected by one-off charges
- Forward looking P/E – actively used by Wall Street analysts; forward
looking – avoids problems with different fiscal years
Market cap. / book value
- Appropriate for financial institutions – Australian trading bank
multiples in particular
Firm value / total assets
- Useful when assessing utilities and other fixed-asset based
companies
24
Source: J.P. Morgan

<details>
<summary>Slide screenshot (PDF page 24)</summary>

![](pic/0209_lecture6_firm_valuation/page-24.png)

</details>


## Page 25 — Tesla Comparable Firms

### Slide content

Tesla Comparable Firms
25
Pure-play EV Manufacturers:

<details>
<summary>Slide screenshot (PDF page 25)</summary>

![](pic/0209_lecture6_firm_valuation/page-25.png)

</details>


## Page 26 — Tesla Comparable Firms

### Slide content

Tesla Comparable Firms
Technology / Autonomy Adjacent Firms:

<details>
<summary>Slide screenshot (PDF page 26)</summary>

![](pic/0209_lecture6_firm_valuation/page-26.png)

</details>


## Page 27 — Comparable Company Analysis

### Slide content

Comparable Company Analysis
27
更换为Tesla的Comparable Company 结果

<details>
<summary>Slide screenshot (PDF page 27)</summary>

![](pic/0209_lecture6_firm_valuation/page-27.png)

</details>


### Comparable-company practice (optional)

The slide suggests replacing the template with Tesla’s comparables.
Below is a compact `akshare` workflow for a handful of peer valuation metrics.


In [12]:
import akshare as ak
import time

PEERS = ['TSLA', 'GM', 'F', 'TM', 'NVDA', 'AAPL', 'GOOGL']

CACHE_PATH = DATA_DIR / 'peer_snapshot.csv'

# Baidu valuation indicators (ak.stock_us_valuation_baidu)
IND_MCAP = '总市值'
IND_PE = '市盈率(TTM)'
IND_PB = '市净率'
IND_PCF = '市现率'
PERIOD = '近一年'
ANNUAL = '年报'

def with_retry(fn, retries=3, backoff=0.6):
    last = None
    for k in range(retries):
        try:
            return fn()
        except Exception as e:
            last = e
            time.sleep(backoff * (k + 1))
    raise last

def latest_value(df):
    # akshare returns columns: date, value
    return float(df.iloc[-1]['value'])

rows = []
peer_snapshot = pd.DataFrame()
if RUN_ONLINE:
    for tkr in PEERS:
        try:
            mcap_yi = latest_value(with_retry(lambda: ak.stock_us_valuation_baidu(tkr, IND_MCAP, PERIOD)))
            pe_ttm = latest_value(with_retry(lambda: ak.stock_us_valuation_baidu(tkr, IND_PE, PERIOD)))
            pb = latest_value(with_retry(lambda: ak.stock_us_valuation_baidu(tkr, IND_PB, PERIOD)))
            pcf = latest_value(with_retry(lambda: ak.stock_us_valuation_baidu(tkr, IND_PCF, PERIOD)))

            # Unit note: Baidu reports 总市值 in '亿' (1e8). Convert to USD dollars.
            market_cap = mcap_yi * 1e8

            # Latest close from Sina (ak.stock_us_daily)
            px = with_retry(lambda: ak.stock_us_daily(symbol=tkr, adjust=''))
            close = float(px.iloc[-1]['close'])

            # Revenue (latest annual) from Eastmoney indicator table
            fin = with_retry(lambda: ak.stock_financial_us_analysis_indicator_em(symbol=tkr, indicator=ANNUAL))
            fin_latest = fin.sort_values('REPORT_DATE').iloc[-1]
            revenue = float(fin_latest['OPERATE_INCOME'])
            currency = str(fin_latest.get('CURRENCY_ABBR', ''))

            mcap_to_rev = (market_cap / revenue) if currency == 'USD' and revenue else np.nan

            rows.append(
                {
                    'Ticker': tkr,
                    'Close': close,
                    'Market cap ($)': market_cap,
                    'P/E (TTM)': pe_ttm,
                    'P/B': pb,
                    'P/CF': pcf,
                    'Revenue (latest)': revenue,
                    'Currency': currency,
                    'MktCap/Revenue (USD only)': mcap_to_rev,
                }
            )
        except Exception as e:
            rows.append({'Ticker': tkr, 'Error': f'{type(e).__name__}: {e}'})
    peer_snapshot = pd.DataFrame(rows).set_index('Ticker') if rows else pd.DataFrame()
    if not peer_snapshot.empty:
        peer_snapshot.to_csv(CACHE_PATH)
        print(f'Cached peer snapshot to: {CACHE_PATH.resolve()}')
else:
    if CACHE_PATH.exists():
        peer_snapshot = pd.read_csv(CACHE_PATH, index_col=0)
        print(f'Loaded cached peer snapshot from: {CACHE_PATH.resolve()}')
    else:
        print('Skipping akshare fetch (RUN_ONLINE=False) and no cache found.')

peer_snapshot


Loaded cached peer snapshot from: C:\Users\Jinquan Ye\OneDrive - Duke University\Research\teaching\K10_calculation\materials\data\peer_snapshot.csv


,Close,Market cap ($),P/E (TTM),P/B,P/CF,Revenue (latest),Currency,MktCap/Revenue (USD only)
Ticker,,,,,,,,
TSLA,411.11,1.542662e+12,406.60,18.77,98.35,9.482700e+10,USD,16.268173
GM,84.24,7.615000e+10,28.24,1.25,3.05,1.850190e+11,USD,0.411579
F,13.80,5.498600e+10,11.68,1.22,2.73,1.849920e+11,USD,0.297234
TM,244.22,3.183010e+11,12.71,1.28,10.23,4.803670e+13,JPY,NaN
NVDA,185.41,4.505463e+12,45.42,37.90,54.59,1.304970e+11,USD,34.525414
AAPL,278.12,4.083119e+12,34.67,46.37,30.41,4.161610e+11,USD,9.811393
GOOGL,322.86,3.905637e+12,29.55,9.40,23.97,4.028360e+11,USD,9.695352


In [13]:
from IPython.display import display

if peer_snapshot.empty or peer_snapshot.dropna(how='all').empty:
    print('No online data to summarize. Set RUN_ONLINE=True to fetch via akshare.')
else:
    numeric = peer_snapshot.select_dtypes(include='number')
    if numeric.empty:
        print('No numeric columns to summarize (all fetches failed).')
    else:
        summary = numeric.describe(percentiles=[0.5]).loc[['mean', '50%']]
        display(summary)


,Close,Market cap ($),P/E (TTM),P/B,P/CF,Revenue (latest),MktCap/Revenue (USD only)
mean,219.965714,2.069474e+12,81.267143,16.598571,31.904286,7.064434e+12,11.834858
50%,244.220000,1.542662e+12,29.550000,9.400000,23.970000,1.850190e+11,9.753373
